In [1]:
# Get helper functions file
import os

if not os.path.exists("helper_functions.py"):
    !wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
else:
    print("[INFO] 'helper_functions.py' already exists, skipping download.")

--2023-12-11 06:55:36--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-12-11 06:55:36 (41.8 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [ ]:
from helper_functions import create_tensorboard_callback , plot_loss_curves,compare_historys

In [1]:
import tensorflow_datasets as tfds


In [2]:
datasets_list = tfds.list_builders()
print("food101" in datasets_list)

True


In [ ]:
(train_data,test_data),ds_info = tfds.load(name = "food101",
                                           shuffle_files=True,
                                           split=["train","validation"],
                                           as_supervised = True,
                                           with_info = True)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
ds_info.features

In [ ]:
class_names = ds_info.features["label"].names
class_names[:10]

In [ ]:
train_one_sample = train_data.take(1)

In [ ]:
for image, label in train_one_sample:
  print(f'''
  Image shape:{image.shape},
  image datatype{image.dtype},
  target_ claass{label},
  class name {class_names[label.numpy()]}''')

In [ ]:
import tensorflow as tf
tf.reduce_min(image),tf.reduce_max(image)

In [ ]:
#preprocessing functions for our data
def preprocess_img(image,label,img_shape = 224):
  """converts image datatype from uint8 to float32 and image size to imgshape
  """
  image = tf.image.resize(image,[img_shape,img_shape])
  return tf.cast(image,tf.float32),label


In [20]:
preprocessed_img =  preprocess_img(image,label)[0]
print(f"Image before{image[:2]} shape {image.shape }  datatype {image.dtype}/n")
print(f"image after preprocessing {preprocessed_img[:2]} and the shape is {preprocessed_img.shape} and the datatype is {preprocessed_img.dtype}")

Image before[[[189 165 121]
  [185 161 117]
  [181 157 113]
  ...
  [111  72  33]
  [103  64  25]
  [ 94  55  16]]

 [[186 162 118]
  [183 159 115]
  [180 156 112]
  ...
  [122  83  40]
  [113  74  31]
  [105  66  23]]] shape (512, 512, 3)  datatype <dtype: 'uint8'>/n
image after preprocessing [[[184.91327  160.91327  116.91326 ]
  [180.35715  156.35715  112.35714 ]
  [183.71428  160.71428  118.71429 ]
  ...
  [147.5915   108.66289   69.08129 ]
  [123.576454  84.576454  44.198887]
  [106.44359   67.44359   25.872158]]

 [[174.66837  150.66837  106.668365]
  [175.14285  151.14285  107.14286 ]
  [177.80103  153.22958  112.015305]
  ...
  [155.61722  119.545784  68.443695]
  [145.1989   108.19381   53.423363]
  [134.37747   98.30605   40.73462 ]]] and the shape is (224, 224, 3) and the datatype is <dtype: 'float32'>


In [ ]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
#batch and prepare datasets
# Map preprocessing function to training data (and paralellize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map prepreprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
# Turn test data into batches (don't need to shuffle)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)

In [11]:
train_data,test_data


(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>)

In [14]:
#creating modelling callbacks
from helper_functions import create_tensorboard_callback
checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor = "val_accuracy",
                                                      save_best_only = True,
                                                      save_weight_only = True,
                                                      verbose = 1)

In [12]:
# mixed precision
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [ ]:

from tensorflow.keras import layers

input_shape = (224, 224, 3)
base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B0(include_top=False)
base_model.trainable = False
inputs = layers.Input(shape=input_shape, name="input_layer")
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs, outputs)

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [ ]:
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Functio  (None, None, None, 1280   5919312   
 nal)                        )                                   
                                                                 
 pooling_layer (GlobalAvera  (None, 1280)              0         
 gePooling2D)                                                    
                                                                 
 dense_8 (Dense)             (None, 101)               129381    
                                                                 
 softmax_float32 (Activatio  (None, 101)               0         
 n)                                                              
                                                           

In [ ]:
for layer in model.layers[1].layers:
  print(layer.name,layer.trainable)

input_10 False
rescaling_10 False
normalization_9 False
stem_conv False
stem_bn False
stem_activation False
block1a_project_conv False
block1a_project_bn False
block1a_project_activation False
block2a_expand_conv False
block2a_expand_bn False
block2a_expand_activation False
block2a_project_conv False
block2a_project_bn False
block2b_expand_conv False
block2b_expand_bn False
block2b_expand_activation False
block2b_project_conv False
block2b_project_bn False
block2b_drop False
block2b_add False
block3a_expand_conv False
block3a_expand_bn False
block3a_expand_activation False
block3a_project_conv False
block3a_project_bn False
block3b_expand_conv False
block3b_expand_bn False
block3b_expand_activation False
block3b_project_conv False
block3b_project_bn False
block3b_drop False
block3b_add False
block4a_expand_conv False
block4a_expand_bn False
block4a_expand_activation False
block4a_dwconv2 False
block4a_bn False
block4a_activation False
block4a_se_squeeze False
block4a_se_reshape False
b

In [ ]:
history_101_food = model.fit(train_data,
                             epochs = 10,
                             steps_per_epoch = len(train_data),
                             validation_data = test_data,
                             validation_steps = len(test_data),
                             callbacks = [create_tensorboard_callback(dir_name = "training_logs",
                                                                      experiment_name = "efficientnetb0101"),
                                          model_checkpoint])


Saving TensorBoard log files to: training_logs/efficientnetb0101/20231207-092441
Epoch 1/10
2366/2368 [============================>.] - ETA: 0s - loss: 0.6649 - accuracy: 0.8253
Epoch 1: val_accuracy improved from -inf to 0.73438, saving model to model_checkpoints/cp.ckpt
2368/2368 [==============================] - 282s 118ms/step - loss: 0.6648 - accuracy: 0.8253 - val_loss: 0.9761 - val_accuracy: 0.7344
Epoch 2/10
2368/2368 [==============================] - ETA: 0s - loss: 0.6407 - accuracy: 0.8305
Epoch 2: val_accuracy improved from 0.73438 to 0.73731, saving model to model_checkpoints/cp.ckpt
2368/2368 [==============================] - 261s 109ms/step - loss: 0.6407 - accuracy: 0.8305 - val_loss: 0.9783 - val_accuracy: 0.7373
Epoch 3/10
2368/2368 [==============================] - ETA: 0s - loss: 0.6191 - accuracy: 0.8374
Epoch 3: val_accuracy did not improve from 0.73731
2368/2368 [==============================] - 236s 99ms/step - loss: 0.6191 - accuracy: 0.8374 - val_loss: 0

In [20]:
from tensorflow import keras
data_augmentation = keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.15),
  layers.RandomZoom(0.12),
  layers.RandomHeight(0.12),
  layers.RandomWidth(0.12),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNet
], name ="data_augmentation")

In [23]:
from tensorflow.keras import layers
def create_model():
  input_shape = (224, 224, 3)
  base_model = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(include_top=False)
  base_model.trainable = False
  inputs = layers.Input(shape=input_shape, name="input_layer")
  x = data_augmentation(inputs)
  x = base_model(inputs, training=False)
  x = layers.GlobalAveragePooling2D(name="pooling_layer")(x)
  x = layers.Dense(101)(x)
  outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
  model = tf.keras.Model(inputs, outputs)

  model.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
  return model

In [ ]:
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model

790/790 [==============================] - 54s 68ms/step - loss: 1.0523 - accuracy: 0.7268


[1.0522972345352173, 0.7268118858337402]

In [ ]:
created_model = create_model()
created_model.compile(loss="sparse_categorical_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

# 3. Load the saved weights
created_model.load_weights(checkpoint_path)

# 4. Evaluate the model with loaded weights
results_created_model_with_loaded_weights = created_model.evaluate(test_data)

52606240/52606240 [==============================] - 0s 0us/step


ValueError: ignored

In [ ]:
save_dir = "/content/drive/Othercomputers/My Laptop/code/MacDeepL/food101/advance07_efficientnetb3_feature_extract_model_mixed_precision/"
os.makedirs(save_dir)
model.save(save_dir)

KeyboardInterrupt: ignored

In [24]:
loaded_gs_model = create_model()
loaded_gs_model.compile(loss="sparse_categorical_crossentropy", # sparse_categorical_crossentropy for labels that are *not* one-hot
                        optimizer=tf.keras.optimizers.Adam(0.0001), # 10x lower learning rate than the default
                        metrics=["accuracy"])


In [25]:
for layer in loaded_gs_model.layers:
    layer.trainable = True
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy)

input_layer True float32 <Policy "float32">
efficientnetv2-b3 True float32 <Policy "mixed_float16">
pooling_layer True float32 <Policy "mixed_float16">
dense True float32 <Policy "mixed_float16">
softmax_float32 True float32 <Policy "float32">


In [26]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                  patience=5)

checkpoint_path = "fine_tune_advance_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss",
                                                      verbose = 0)

In [27]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2,
                                                 patience=2,
                                                 verbose=1,
                                                 min_lr=1e-7)

In [28]:
history_101_food_classes_all_data_fine_tune = loaded_gs_model.fit(train_data,
                                                        epochs=100,
                                                        steps_per_epoch=len(train_data),
                                                        validation_data=test_data,
                                                        validation_steps=int(0.15 * len(test_data)),
                                                        callbacks=[create_tensorboard_callback("training_logs", "efficientb0_101_classes_all_data_fine_tuning"),
                                                                   model_checkpoint,
                                                                   early_stopping,
                                                                   reduce_lr])

Saving TensorBoard log files to: training_logs/efficientb0_101_classes_all_data_fine_tuning/20231211-072012
Epoch 1/100
2368/2368 [==============================] - ETA: 0s - loss: 2.8794 - accuracy: 0.4002
Epoch 1: val_loss improved from inf to 1.91578, saving model to fine_tune_advance_checkpoints/
2368/2368 [==============================] - 286s 111ms/step - loss: 2.8794 - accuracy: 0.4002 - val_loss: 1.9158 - val_accuracy: 0.5776 - lr: 1.0000e-04
Epoch 2/100
2367/2368 [============================>.] - ETA: 0s - loss: 1.8733 - accuracy: 0.5626
Epoch 2: val_loss improved from 1.91578 to 1.52845, saving model to fine_tune_advance_checkpoints/
2368/2368 [==============================] - 244s 102ms/step - loss: 1.8733 - accuracy: 0.5627 - val_loss: 1.5284 - val_accuracy: 0.6361 - lr: 1.0000e-04
Epoch 3/100
2367/2368 [============================>.] - ETA: 0s - loss: 1.6222 - accuracy: 0.6053
Epoch 3: val_loss improved from 1.52845 to 1.38603, saving model to fine_tune_advance_checkpo

KeyboardInterrupt: ignored

In [30]:
food101hope = create_model()
food101hope.compile(loss="sparse_categorical_crossentropy",
                      optimizer=tf.keras.optimizers.Adam(),
                      metrics=["accuracy"])

# 3. Load the saved weights
food101hope.load_weights(checkpoint_path)

# 4. Evaluate the model with loaded weights
results_created_model_with_loaded_weights = food101hope.evaluate(test_data)
results_created_model_with_loaded_weights

790/790 [==============================] - 71s 83ms/step - loss: 0.9943 - accuracy: 0.7343


[0.9943013787269592, 0.7343366146087646]

In [ ]:
from tensorflow.keras.models import load_model
#food101hope.save("/content/drive/Othercomputers/My Laptop/code/MacDeepL/food101/finaladvance07_efficientnetb0_fine_tuned_101_classes_augmentation/")
model = load_model("/content/drive/Othercomputers/My Laptop/code/MacDeepL/food101/07_efficientnetb0_fine_tuned_101_classes_mixed_precision")
print(model.evaluate(test_data))

In [ ]:
from tensorflow.keras.models import load_model
#food101hope.save("/content/drive/Othercomputers/My Laptop/code/MacDeepL/food101/finaladvance07_efficientnetb0_fine_tuned_101_classes_augmentation/")
modelff = load_model("/content/drive/Othercomputers/My Laptop/code/MacDeepL/food101/finaladvance07_efficientnetb0_fine_tuned_101_classes_augmentation")
print(modelff.evaluate(test_data))

In [ ]:
from tensorflow.keras.models import load_model
#food101hope.save("/content/drive/Othercomputers/My Laptop/code/MacDeepL/food101/finaladvance07_efficientnetb0_fine_tuned_101_classes_augmentation/")
modelnoft = load_model("/content/drive/Othercomputers/My Laptop/code/MacDeepL/food101/07_efficientnetb0_feature_extract_model_mixed_precision")
print(modelnoft.evaluate(test_data))